In [82]:
import xarray as xr
import numpy as np
import sys
from pathlib import Path
import pandas as pd
import geopandas as gpd
from scipy.stats import gamma, norm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import fisk, norm


# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Configuration

In [17]:
# Configuration
config = {
    'input_data_dir': project_root / 'data' / 'output_data' / 'merged_2d'/'1980_2024'/'SPEI'/'bow_combined_SPEI_data.csv',
    'shapefile_dir': project_root / 'data' / 'input_data' / 'shapefiles'/'BowRiverBasin'/'Bow_elevation_combined.shp',
    'merged_output_dir': project_root / 'data' / 'output_data' / 'merged_2d'/'1980_2024'/'SPEI',
    'plots': project_root / 'data' / 'output_plots' / '1980_2024'/'SPEI',
}

Open data for analysis

In [14]:
# Open shape file
bow_basin = gpd.read_file(config['shapefile_dir'])
display(bow_basin.head())

,PROVCD_1,VALDATE,EDITION,DATASETNAM,VERSION,COMPLEVEL,WSCMDA,WSCSDA,WSCSSDA,FDA,...,WSCSSDANAM,min,max,mean,count,std,median,PROVCD_2,elev_class,geometry
0,AB,20070208,1,05BM000,0,NHN-CL1,05,05B,05BM,05BM,...,Lower Bow - Crowfoot,776.0,1177.0,953.688214,9897452,71.132927,944.0,None,500_1000m,"POLYGON ((-112.58577 51.23024, -112.58519 51.2..."
1,AB,20070130,1,05BN000,0,NHN-CL1,05,05B,05BN,05BN,...,Lower Bow - Mouth,700.0,998.0,772.376015,9312616,32.852467,764.0,None,500_1000m,"POLYGON ((-112.4962 50.71881, -112.49566 50.71..."
2,AB,20070226,1,05BH000,0,NHN-CL1,05,05B,05BH,05BH,...,Central Bow - Jumpingpond,1038.0,2479.0,1259.913722,4121228,176.434021,1227.0,None,1000_1500m,"POLYGON ((-113.93263 51.12563, -113.93288 51.1..."
3,AB,20070228,1,05BK000,0,NHN-CL1,05,05B,05BK,05BK,...,Fish (Alta.),981.0,1777.0,1226.170423,1151005,126.086335,1194.0,None,1000_1500m,"POLYGON ((-114.10188 50.95504, -114.0975 50.95..."
4,AB,20070302,1,05BE000,0,NHN-CL1,05,05B,05BE,05BE,...,Upper Bow - Policeman,1146.0,3054.0,1601.837836,1655101,376.052387,1437.0,None,1500_2000m,"POLYGON ((-114.72215 51.33033, -114.72235 51.3..."


In [26]:
SPEI_data = pd.read_csv(config['input_data_dir'])
SPEI_data.set_index('time', inplace=True)

display(SPEI_data.head())

,Grid_id,lon,lat,Precipitation,Mean_Temp,Humidity,Flux,Surface_Pressure,Vwind,Uwind,Elevation,Elevation_Category
time,,,,,,,,,,,,
1980-10-10,1,-116.1380,51.3346,0.009879,-3.783902,0.002236,3049.015615,792.259058,0.311295,-0.084560,213.40561,2000_2500m
1982-07-19,1,-116.1380,51.3346,2.765854,8.561398,0.007391,4451.734360,781.952682,0.568268,1.897212,213.40561,2000_2500m
1983-10-12,2,-116.1875,51.4191,0.023846,-3.537311,0.002839,3352.359415,793.773767,-0.336093,-0.022384,211.10257,2000_2500m
1980-12-11,2,-116.1875,51.4191,4.578907,-7.179487,0.002436,472.984375,778.646749,0.921281,2.757770,211.10257,2000_2500m
1981-07-21,2,-116.1875,51.4191,5.400473,9.413436,0.008504,4196.250000,789.834761,0.053097,0.938094,211.10257,2000_2500m


In [29]:
# Convert index to datetime if not already
SPEI_data.index = pd.to_datetime(SPEI_data.index)

# add season year column
def get_season_year(date):
    if date.month >= 10:
        return date.year 
    else:
        return date.year-1
SPEI_data['season_year'] = SPEI_data.index.to_series().apply(get_season_year)


display(SPEI_data)


,Grid_id,lon,lat,Precipitation,Mean_Temp,Humidity,Flux,Surface_Pressure,Vwind,Uwind,Elevation,Elevation_Category,season_year
time,,,,,,,,,,,,,
1980-10-10,1,-116.1380,51.3346,0.009879,-3.783902,0.002236,3049.015615,792.259058,0.311295,-0.084560,213.40561,2000_2500m,1980
1983-10-12,2,-116.1875,51.4191,0.023846,-3.537311,0.002839,3352.359415,793.773767,-0.336093,-0.022384,211.10257,2000_2500m,1983
1980-12-11,2,-116.1875,51.4191,4.578907,-7.179487,0.002436,472.984375,778.646749,0.921281,2.757770,211.10257,2000_2500m,1980
1982-10-29,2,-116.1875,51.4191,1.777088,-8.325263,0.002016,2569.312515,774.911829,-0.176696,1.177495,211.10257,2000_2500m,1982
1982-01-18,2,-116.1875,51.4191,1.890419,-16.744781,0.001073,1106.484380,765.437045,-0.148752,0.036916,211.10257,2000_2500m,1981
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-22,257,-111.6202,50.0960,0.011239,-1.576073,0.003107,2602.781245,927.793413,0.055882,3.991391,75.06684,500_1000m,2023
2024-03-19,257,-111.6202,50.0960,0.000257,4.113504,0.003839,3415.656250,931.615300,-3.568600,-2.436361,75.06684,500_1000m,2023
2024-01-06,257,-111.6202,50.0960,0.135581,-2.378316,0.002423,1178.328135,918.650769,2.518928,3.470230,75.06684,500_1000m,2023


# Calculated PET - Thornthwaite method

In [55]:
def thornthwaite_pet(T_monthly, lat):
    """
    T_monthly: xarray DataArray [time, y, x] in °C (monthly mean)
    lat: xarray DataArray [y, x] in degrees
    Returns PET in mm/month
    """

    # Days per month
    days_in_month = T_monthly["time"].dt.days_in_month

    # Monthly heat index i
    T_pos = T_monthly.where(T_monthly > 0, 0)
    i = (T_pos / 5.0) ** 1.514

    # Annual heat index I (per year)
    I = i.groupby("time.year").sum("time")

    # Thornthwaite exponent a
    a = (6.75e-7 * I**3
         - 7.71e-5 * I**2
         + 1.792e-2 * I
         + 0.49239)

    # Broadcast a back to monthly
    a_m = a.sel(year=T_monthly["time.year"]).drop("year")

    # Latitude in radians
    phi = np.deg2rad(lat)

    # Day of year (mid-month)
    J = T_monthly["time"].dt.dayofyear

    delta = 0.409 * np.sin(2 * np.pi * J / 365.0 - 1.405)
    omega_s = np.arccos(-np.tan(phi) * np.tan(delta))

    # Mean day length (hours)
    L = (24.0 / np.pi) * omega_s

    # Thornthwaite PET
    PET = (16.0 *
           (L / 12.0) *
           (days_in_month / 30.0) *
           (10.0 * T_pos / I.sel(year=T_monthly["time.year"]).drop("year")) ** a_m)

    return PET

In [100]:
def thornthwaite_pet(T_monthly, lat, eps=1e-12):
    """
    Thornthwaite PET in mm/month.

    T_monthly: xarray DataArray [time, y, x] monthly mean temperature (°C)
    lat:       xarray DataArray [y, x] latitude (degrees)
    """

    # Ensure temperature > 0°C only (Thornthwaite convention)
    T_pos = T_monthly.where(T_monthly > 0, 0.0)

    # Days per month (time)
    days_in_month = T_monthly["time"].dt.days_in_month

    # Monthly heat index i_m
    i_m = (T_pos / 5.0) ** 1.514

    # Annual heat index I (per year) [y, x]
    I = i_m.groupby("time.year").sum("time")

    # Thornthwaite exponent a(I) [y, x]
    a = (6.75e-7 * I**3
         - 7.71e-5 * I**2
         + 1.792e-2 * I
         + 0.49239)

    # Broadcast I and a back to monthly [time, y, x]
    year = T_monthly["time"].dt.year
    I_m = I.sel(year=year).drop("year")
    a_m = a.sel(year=year).drop("year")

    # Avoid divide-by-zero / nonsense when I ~ 0
    I_m = I_m.where(I_m > eps)

    # Latitude in radians, broadcast to [time, y, x]
    phi = np.deg2rad(lat).broadcast_like(T_monthly)

    # Use mid-month day-of-year for each timestamp
    mid_time = T_monthly["time"] - xr.conventions.times.decode_cf_datetime(
        xr.DataArray(np.zeros(T_monthly.sizes["time"], dtype=int), dims="time", coords={"time": T_monthly["time"]}),
        units="days since 1970-01-01"
    )  # (noop placeholder; see simpler option below)

    # Day-of-year at ~mid-month: take month start + 14 days
    month_start = T_monthly["time"].dt.floor("D") - (T_monthly["time"].dt.day - 1).astype("timedelta64[D]")
    mid_month = month_start + np.timedelta64(14, "D")
    J = mid_month.dt.dayofyear

    # Solar declination (radians)
    delta = 0.409 * np.sin(2 * np.pi * J / 365.0 - 1.39)
    delta = delta.broadcast_like(T_monthly)

    # Sunset hour angle
    omega_s = np.arccos((-np.tan(phi) * np.tan(delta)).clip(min=-1, max=1))

    # Mean day length (hours)
    L = (24.0 / np.pi) * omega_s

    # Thornthwaite PET (mm/month)
    PET = 16.0 * (L / 12.0) * (days_in_month / 30.0) * ((10.0 * T_pos / I_m) ** a_m)

    # Ensure PET=0 when T<=0
    PET = PET.where(T_monthly > 0, 0.0)

    PET.name = "PET_thornthwaite"
    return PET

In [102]:
# Calculate monthly mean temperature for each grid point (per Grid_id) as an xarray DataArray
temp_wide = SPEI_data.reset_index().set_index(['time', 'Grid_id'])['Mean_Temp'].unstack('Grid_id')

T_da = xr.DataArray(
	temp_wide.to_numpy(),
	dims=['time', 'Grid_id'],
	coords={'time': temp_wide.index, 'Grid_id': temp_wide.columns},
	name='Mean_Temp'
)

# Monthly mean over each calendar month
T_monthly = T_da.resample(time='1M').mean()
T_monthly.name = 'Mean_Temp_monthly'

# Get latitude per Grid_id as an xarray DataArray aligned to the temperature grid
lat_series = SPEI_data[['Grid_id', 'lat']].drop_duplicates('Grid_id').set_index('Grid_id')['lat']
lat = xr.DataArray(lat_series, dims=['Grid_id'], coords={'Grid_id': lat_series.index}, name='lat')

pet_th = thornthwaite_pet(T_monthly, lat,eps=1e-12)
pet_th.name = 'PET'

# keep precipitation , season_year, Elevation_Category columns from SPEI_data



pet_th = pet_th.to_dataframe().reset_index()

display(pet_th)

c:\Users\walimunige.rupasingh\OneDrive - University of Calgary\Documents\GitHub\snowdroughtindex-main\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_24316\1274036726.py:29: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  I_m = I.sel(year=year).drop("year")
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_24316\1274036726.py:30: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  a_m = a.sel(year=year).drop("year")


AttributeError: module 'xarray.conventions' has no attribute 'times'

In [ ]:
# Calculate monthly precipitation for each season year and grid point
monthly_precip = (
    SPEI_data
    .groupby(['Grid_id', 'season_year','Elevation_Category', pd.Grouper(key='time', freq='M')])['Precipitation']
    .sum()
    .reset_index()
)


display(monthly_precip)

C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_24316\2271054864.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(['Grid_id', 'season_year','Elevation_Category', pd.Grouper(freq='M')])['Precipitation']


,Grid_id,season_year,Elevation_Category,time,Precipitation
0,1,1979,2000_2500m,1980-01-31,36.775476
1,1,1979,2000_2500m,1980-02-29,22.427567
2,1,1979,2000_2500m,1980-03-31,40.851821
3,1,1979,2000_2500m,1980-04-30,10.105498
4,1,1979,2000_2500m,1980-05-31,129.108904
...,...,...,...,...,...
92515,257,2023,500_1000m,2024-04-30,21.840489
92516,257,2023,500_1000m,2024-05-31,89.019734
92517,257,2024,500_1000m,2024-10-31,9.685615
92518,257,2024,500_1000m,2024-11-30,21.551023


In [91]:
# combine PET and precipitation dataframes
SPEI_combined = pet_th.merge(monthly_precip, on=['time', 'Grid_id'], how='inner')
display(SPEI_combined)

,Grid_id,time,PET,season_year,Elevation_Category,Precipitation
0,1,1980-01-31,0.000000,1979,2000_2500m,36.775476
1,1,1980-02-29,0.000000,1979,2000_2500m,22.427567
2,1,1980-03-31,0.000000,1979,2000_2500m,40.851821
3,1,1980-04-30,0.000000,1979,2000_2500m,10.105498
4,1,1980-05-31,177.214024,1979,2000_2500m,129.108904
...,...,...,...,...,...,...
92515,257,2024-04-30,73.634853,2023,500_1000m,21.840489
92516,257,2024-05-31,117.518468,2023,500_1000m,89.019734
92517,257,2024-10-31,61.634682,2024,500_1000m,9.685615
92518,257,2024-11-30,0.000000,2024,500_1000m,21.551023


In [92]:
# Calculate water balance D = P - PET
SPEI_combined['D'] = SPEI_combined['Precipitation'] - SPEI_combined['PET']

display(SPEI_combined)

,Grid_id,time,PET,season_year,Elevation_Category,Precipitation,D
0,1,1980-01-31,0.000000,1979,2000_2500m,36.775476,36.775476
1,1,1980-02-29,0.000000,1979,2000_2500m,22.427567,22.427567
2,1,1980-03-31,0.000000,1979,2000_2500m,40.851821,40.851821
3,1,1980-04-30,0.000000,1979,2000_2500m,10.105498,10.105498
4,1,1980-05-31,177.214024,1979,2000_2500m,129.108904,-48.105120
...,...,...,...,...,...,...,...
92515,257,2024-04-30,73.634853,2023,500_1000m,21.840489,-51.794364
92516,257,2024-05-31,117.518468,2023,500_1000m,89.019734,-28.498734
92517,257,2024-10-31,61.634682,2024,500_1000m,9.685615,-51.949067
92518,257,2024-11-30,0.000000,2024,500_1000m,21.551023,21.551023


In [96]:
# Calculate monthly average precipitation for each Elevation_Category, season_year, and month
SPEI_combined['month'] = SPEI_combined['time'].dt.month

monthly_avg = (
    SPEI_combined
    .groupby(['Elevation_Category', 'season_year', 'month'])['D']
    .mean()
    .unstack(level=0)  # Elevation_Category as columns
)

# Rename columns to match the format "2000-2500m_P"
monthly_avg.columns = [f"{col}_D" for col in monthly_avg.columns]

#drop seasonYear 1979 and 2024
monthly_avg = monthly_avg[(monthly_avg.index.get_level_values('season_year') != 1979) & (monthly_avg.index.get_level_values('season_year') != 2024)]        

display(monthly_avg.head(10))

1000_1500m_D  1500_2000m_D  2000_2500m_D  500_1000m_D
season_year month                                                       
1980        1          6.245697      9.145041     11.221143     8.226255
            2          8.437483     13.018755     28.045795     4.767321
            3         -1.796954      3.986165     14.435462    20.336954
            4        -65.395525    -33.081371     27.394831   -74.238574
            5          3.467370     16.034033   -102.507525   -34.664472
            10       -30.144784    -19.662122      8.267500   -19.100106
            11        16.905235     30.893936     52.304052    10.509811
            12        24.672522     39.349471     94.355993    27.466716
1981        1         13.760274     15.885581     32.796716    18.841189
            2          9.288835     13.259168     30.915211    11.693719

In [97]:
# Group by season and sum the precipitation for each column
seasonal_D = monthly_avg.groupby('season_year').sum(numeric_only=True)

display(seasonal_D)

,1000_1500m_D,1500_2000m_D,2000_2500m_D,500_1000m_D
season_year,,,,
1980,-37.608956,59.683907,133.517252,-56.696095
1981,-31.975561,4.519271,0.003042,-42.963878
1982,-92.666295,-17.363811,6.707379,-116.426922
1983,-138.648221,-67.696984,-25.660318,-153.142376
1984,-120.399328,-23.437127,18.644082,-86.792122
1985,-71.034887,18.810311,123.944288,-117.854422
1986,-160.998941,-95.449221,40.170000,-176.327541
1987,-183.353380,-86.635559,30.242252,-200.017551
1988,-90.201064,-27.740935,-0.609990,-105.352226


# Calculate SPEI

In [98]:
# Calculate SPEI

def calculate_spei_loglogistic(D_series, eps=1e-12):
    """
    SPEI using 3-parameter log-logistic (Fisk) distribution.
    D_series: pandas Series of D = P - PET (can be negative)
    Returns: pandas Series of SPEI (standard normal)
    """
    s = pd.Series(D_series).astype(float)

    # keep finite values only for fitting
    fit_data = s[np.isfinite(s.values)]
    if fit_data.size < 5 or np.nanstd(fit_data.values) == 0:
        return pd.Series(np.nan, index=s.index)

    # Fit 3-parameter log-logistic (shape=c, loc, scale)
    c, loc, scale = fisk.fit(fit_data.values)

    # CDF for all values (including negatives)
    cdf = fisk.cdf(s.values, c, loc=loc, scale=scale)

    # Avoid inf from norm.ppf at 0 or 1
    cdf = np.clip(cdf, eps, 1 - eps)

    spei = norm.ppf(cdf)
    return pd.Series(spei, index=s.index)

spei_results = {}
for col in seasonal_D.columns:
    spei_results[col.replace('_D', '_SPEI')] = calculate_spei_loglogistic(seasonal_D[col])

spei_df = pd.DataFrame(spei_results, index=seasonal_D.index)
display(spei_df)



,1000_1500m_SPEI,1500_2000m_SPEI,2000_2500m_SPEI,500_1000m_SPEI
season_year,,,,
1980,0.979272,0.945896,1.066076,1.036492
1981,0.986566,0.860344,0.662363,1.055373
1982,0.886980,0.811548,0.711405,0.923480
1983,0.745911,0.602137,0.208490,0.798979
1984,0.814687,0.795449,0.779602,0.987454
1985,0.928832,0.886528,1.052178,0.919821
1986,0.603955,-4.134706,0.867601,0.648045
1987,-4.132884,0.361790,0.831084,-4.359419
1988,0.892248,0.783161,0.657338,0.949823
